## FINETUNING
but they want money

In [3]:
import openai
import os

# Set your API key (preferably using an environment variable)
openai.api_key = ''
  # Recommended: use environment variable
# Alternative (not recommended):
# openai.api_key = 'your-api-key-here'

# Step 1: Upload the dataset
try:
    file_response = openai.files.create(
        file=open("data.jsonl", "rb"),
        purpose="fine-tune"
    )
    file_id = file_response.id
    print(f"Uploaded file ID: {file_id}")

    # Step 2: Start the fine-tuning job
    fine_tune_response = openai.fine_tuning.jobs.create(
        training_file=file_id,
        model="gpt-3.5-turbo"  # Note: Changed from gpt-4o-mini as that's not a standard model
    )
    fine_tune_job_id = fine_tune_response.id
    print(f"Fine-tune job ID: {fine_tune_job_id}")

except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: [Errno 2] No such file or directory: 'data.jsonl'


## COMPILING LEAN

In [4]:
import os
import subprocess


def lean_compile(state: str):
    workspace_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))  # Navigate up to NNG4 root

    # Create temp file in the correct location
    temp_file_path = os.path.join(workspace_path, "Game", "Levels", "Addition", "temp_theorem.lean")

    with open(temp_file_path, "w") as f:
        f.write(state)

    # First make sure we're in the Lake project directory
    os.chdir(project_dir)


    # Try using lake env to run lean
    result = subprocess.run(
        ['lake', 'env', 'lean', temp_file_path],
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        return False
    else:
        return result.stdout


In [9]:
import os
import subprocess

def lean_compile(state: str):
    workspace_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))  # Navigate up to NNG4 root

    # Create temp file in the correct location
    temp_file_path = os.path.join(workspace_path, "Game", "Levels", "Addition", "temp_theorem.lean")
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(temp_file_path), exist_ok=True)

        # Write the Lean code to a temporary file
        with open(temp_file_path, "w") as f:
            # Explicitly add necessary imports and namespace
            full_content = f"""{state}
end MyNat
"""
            f.write(full_content)

        # Try compiling with explicit project root
        result = subprocess.run(
            ['lake', 'env', 'lean', temp_file_path],
            capture_output=True,
            text=True,
            cwd="./",
            timeout=30
        )
        print(result)

        # Check compilation result
        if result.returncode == 0:
            return False  # Successful compilation
        else:
            # Return detailed error output
            return f"Compilation Error:\n{result.stdout}\n{result.stderr}"

    except subprocess.TimeoutExpired:
        return "Compilation timed out"
    except Exception as e:
        return f"Compilation error: {str(e)}"


In [19]:
tactics = [
    "rw [name_of_theorem]",
    "rw [ \\l name_of_theorem]",
    "induction k with d hd",
    "rfl",
]

theorems = [
    "add_succ",
    "add_zero",
    "add_assoc",
    "add_comm",
    'hd'
]


In [7]:
lean_compile('''import Game.Levels.Addition.L04add_assoc
namespace MyNat
theorem zero_add_thm (n: ℕ) : 0 + n = n := by
  induction n with d hd
  rw [add_zero]
  rfl
  rw [add_succ]
  rw [hd]
  rfl

'''
             )

CompletedProcess(args=['lake', 'env', 'lean', '/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4/Game/Levels/Addition/temp_theorem.lean'], returncode=1, stdout="/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4/Game/Levels/Addition/temp_theorem.lean:1:0: error: unknown package 'Game'\nYou might need to open '/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4' as a workspace in your editor\n/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4/Game/Levels/Addition/temp_theorem.lean:3:31: error: unexpected token '+'; expected ':=', 'where' or '|'\n", stderr='')


"Compilation Error:\n/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4/Game/Levels/Addition/temp_theorem.lean:1:0: error: unknown package 'Game'\nYou might need to open '/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4' as a workspace in your editor\n/Users/raynabhattacharyya/Documents/Berkeley/Research/AIinEd/NNG4/Game/Levels/Addition/temp_theorem.lean:3:31: error: unexpected token '+'; expected ':=', 'where' or '|'\n\n"

In [6]:
import os
import json
from openai import OpenAI


def ask_llm(statement):
    client = OpenAI(api_key='')
    prompt = (
        f"""

        Here are all the possible lean tactics that you can use:
        "rw [name_of_theorem]": "rewrite using a theorem make sure the box brackets exists aroud name_of_theorem",
        "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem, make sure the box brackets exists aroud \l name_of_theorem",
        "induction k with d hd": '''induction on k with d as the base case and hd as the induction step
        the lean compiler automatically creates the goals for the base case and the induction step, and we start by solving the base case
        followed by the induction step''',
        'rfl': "whenever a goal is completed and the LHS and RHS are exactly equal, use rfl to close the goal",
        'add_succ': "theorem that for any natural numbers n and m: n + succ(m) = succ(n + m)",
        'add_zero': "theorem that for any natural number n: n + 0 = n",
        'add_assoc': "theorem that for any natural numbers n, m, and k: n + (m + k) = (n + m) + k",
        'add_comm': "theorem that for any natural numbers n and m: n + m = m + n",


        convert the following natural language steps into lean4 tactics:
        Note: one step only corresponds to one tactic
        Note: CONVERT THE TACTICS EXACTLY HOW THE STUDENT HAS WRITTEN THEM IN THE NATURAL LANGUAGE DO NOT WORRY ABOUT LEAN COMPILATION ERRORS

        {statement}

        give me a list of tactics one after the other note that it is possible that one step may require multiple tactics
        give me the ouput in the format ONLY and no other text:
        tactic_1
        tactic_2
        tactic_3
        ...
        """
        )
    response = client.chat.completions.create(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini", temperature=0, n=1)
    return(response.choices[0].message.content)

In [ ]:
import os
import json
from openai import OpenAI
from pydantic import BaseModel
from enum import Enum
from typing import Optional

class Step_index(BaseModel):
    lean_tactic_template: str
    tactic_input: Optional[str]
    NL: str

class Steps_index(BaseModel):
    steps: list[Step_index]

class Step(BaseModel):
    tactic_template: int
    theorem_name: Optional[int]

class Steps(BaseModel):
    steps: list[Step]

class Error(BaseModel):
    line: int
    tutor: str

def ask_llm_tutor_structured(error_statement, nl_tactics, lean_tactics):
    client = OpenAI(api_key='')
    prompt = (
        f"""
        Here are ALL the lean tactics THAT OUR LEAN NAMESPACE RECONGIZES:
        "rw [name_of_theorem]": "rewrite using a theorem make sure the box brackets exists aroud name_of_theorem",
        "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem make sure the box brackets exists aroud name_of_theorem",
        "induction k with d hd": '''induction on k with d as the base case and hd as the induction step
        . Note when the base case has been proved you must end with the rfl and similarly with the induction step''',
        'rfl': "whenever a goal is completed and the LHS and RHS are exactly equal, use rfl to close the goal",
        'add_succ': "theorem that for any natural numbers n and m: n + succ(m) = succ(n + m)",
        'add_zero': "theorem that for any natural number n: n + 0 = n",
        'add_assoc': "theorem that for any natural numbers n, m, and k: n + (m + k) = (n + m) + k",
        'add_comm': "theorem that for any natural numbers n and m: n + m = m + n",

        A student has made an error in their proof. We converted the NL steps to Lean tactics and tried to compile the code but it failed. Here is the error message:
        {error_statement}
        Write a tutor response to the student explaining the error and how they can fix it. Do not provde the solution, only the explanation of the error and how they can fix it.
        KEEP your response really short and simple.

        here are the steps that the student has written in natural language and the corresponding lean tactics:
        NL_TACTICS: {nl_tactics} ;
        LEAN_TACTICS: {lean_tactics}
        NOTE A STUDENT DOES NOT UNDERSTAND LEAN BUT THEY UNDERSTAND THE MATH
        OF THE TYPE WRITTEN ABOVE IN THE NL_TACTICS
        SO DO NOT USE LEAN TERMINOLOGY IN YOUR RESPONSE BUT USE NATURAL LANGUAGE TACTIC TERMINOLOGY
        """
    )
    response = client.beta.chat.completions.parse(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini-2024-07-18", temperature=0, n=1, response_format=Error)
    return response.choices[0].message.parsed

def ask_llm_structured_2(statement):
    client = OpenAI(api_key='')
    prompt = (
        f"""
        here are all the possible lean tactics that you can PLEASE ONLY USE THESE TACTICS:
        Here are ALL the lean tactics THAT OUR LEAN NAMESPACE RECONGIZES:
        "rw [name_of_theorem]": "rewrite using a theorem make sure the box brackets exists aroud name_of_theorem",
        "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem make sure the box brackets exists aroud name_of_theorem",
        "induction k with d hd": '''induction on k where d represents the base case and hd reprsents the induction step
        . Note when the base case has been proved you must end with the rfl and similarly with the induction step''',
        'rfl': "whenever a goal is completed and the LHS and RHS are exactly equal, use rfl to close the goal",
        'add_succ': "theorem that for any natural numbers n and m: n + succ(m) = succ(n + m)",
        'add_zero': "theorem that for any natural number n: n + 0 = n",
        'add_assoc': "theorem that for any natural numbers n, m, and k: n + (m + k) = (n + m) + k",
        'add_comm': "theorem that for any natural numbers n and m: n + m = m + n",
        'hd': "induction hypothesis",

        YOU CAN ONLY USE ONE OF THE ABOVE TACTICS DO NOT USE ANY OTHER TACTICS

        convert each natural language step into lean tactic indepdantly:
        Note: there is a one-one mapping between steps and tactics
        {statement}
        """
        )
    response = client.beta.chat.completions.parse(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini-2024-07-18", temperature=0, n=1, response_format=Steps_index)
    return response.choices[0].message.parsed

<>:53: SyntaxWarning: invalid escape sequence '\l'
<>:97: SyntaxWarning: invalid escape sequence '\l'
<>:125: SyntaxWarning: invalid escape sequence '\l'
<>:53: SyntaxWarning: invalid escape sequence '\l'
<>:97: SyntaxWarning: invalid escape sequence '\l'
<>:125: SyntaxWarning: invalid escape sequence '\l'
/var/folders/l_/zz2st2097tg79c4ff5gdms5m0000gn/T/ipykernel_50348/3549923470.py:53: SyntaxWarning: invalid escape sequence '\l'
  """
/var/folders/l_/zz2st2097tg79c4ff5gdms5m0000gn/T/ipykernel_50348/3549923470.py:97: SyntaxWarning: invalid escape sequence '\l'
  """
/var/folders/l_/zz2st2097tg79c4ff5gdms5m0000gn/T/ipykernel_50348/3549923470.py:125: SyntaxWarning: invalid escape sequence '\l'
  """


## NL -> Formal Dictionary

In [ ]:
tactic_NL_dict = {
  "rw [name_of_theorem]": "rewrite using a theorem",
  "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem",
  "induction k with d hd": '''induction on k with d as the base case and hd as the induction step
  the lean compiler automatically creates the goals for the base case and the induction step, and we start by solving the base case
  followed by the induction step''',
  'rfl': "whenever a goal is completed and the LHS and RHS are exactly equal, use rfl to close the goal",
  'add_succ': "theorem that for any natural numbers n and m: n + succ(m) = succ(n + m)",
  'add_zero': "theorem that for any natural number n: n + 0 = n",
  'add_assoc': "theorem that for any natural numbers n, m, and k: n + (m + k) = (n + m) + k",
  'add_comm': "theorem that for any natural numbers n and m: n + m = m + n",
}

<>:3: SyntaxWarning: invalid escape sequence '\l'
<>:3: SyntaxWarning: invalid escape sequence '\l'
/var/folders/l_/zz2st2097tg79c4ff5gdms5m0000gn/T/ipykernel_49190/3204633841.py:3: SyntaxWarning: invalid escape sequence '\l'
  "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem",


## Prompt

In [ ]:
def ask_llm_structured(statement):
    client = OpenAI(api_key='')
    prompt = (
        f"""
        Here are ALL the lean tactics THAT OUR LEAN NAMESPACE RECONGIZES:

        For each step in the proof:

        select one of the tactics from the list below:

        THIS IS A LIST OF TACTICS:
        1: "rw [name_of_theorem]": "rewrite using a theorem make sure the box brackets exists aroud name_of_theorem",
        2: "rw [ \l name_of_theorem]": "rewrite using the left-hand side of a theorem make sure the box brackets exists aroud name_of_theorem",
        3: "induction k with d hd": '''induction on k where d represents the base case and hd reprsents the induction step
        . Note when the base case has been proved you must end with the rfl and similarly with the induction step''',
        4: 'rfl': "whenever a goal is completed and the LHS and RHS are exactly equal, use rfl to close the goal",

        THIS IS A LIST OF THEOREMS:
        1: 'add_succ': "theorem that for any natural numbers n and m: n + succ(m) = succ(n + m)",
        2: 'add_zero': "theorem that for any natural number n: n + 0 = n",
        3: 'add_assoc': "theorem that for any natural numbers n, m, and k: n + (m + k) = (n + m) + k",
        4: 'add_comm': "theorem that for any natural numbers n and m: n + m = m + n",
        5: 'hd': "induction hypothesis",

        YOU CANNOT USE A THEOREM BY THEMSELVES YOU MUST USE THEM IN A TACTIC

        return a list of indices of tactics and the theorem names.
        IMPORTANT: If the step doesn't match closely with any of the tactics, return None as the tactic index and None as the theorem name
        If a tactic does not have name_of_theorem, just input none as the theorem name

        convert each natural language step into lean tactic indepdantly:
        Note: there is a one-one mapping between steps and tactics
        {statement}
        """
        )
    response = client.beta.chat.completions.parse(messages=[{"role": "user", "content": prompt}], model="gpt-4o-mini-2024-07-18", temperature=0, n=1, response_format=Steps)
    return response.choices[0].message.parsed


<>:34: SyntaxWarning: invalid escape sequence '\l'
<>:34: SyntaxWarning: invalid escape sequence '\l'
/var/folders/l_/zz2st2097tg79c4ff5gdms5m0000gn/T/ipykernel_50348/2507010694.py:34: SyntaxWarning: invalid escape sequence '\l'
  """


## Correct Solution

In [198]:
steps = """
problem: theorem zero_add_thm (k: ℕ) : 0 + k = k
step1: we start by induction on k
step2: for the base case we show that we have 0 + 0 = 0
step3: hence we have proved the base case
step4: to show that 0 + d + 1 = d + 1 we use n + m + 1 = n + m + 1 substituting n with 0 and m with d
step5: using the induction hypothesis we can subsitute
step6: hence we have proved the k+1 step
"""


## Incorrect Solutions

In [157]:
wrong_steps_1 = """
problem: theorem zero_add_thm (k: ℕ) : 0 + k = k
step1: we begin by induction
step2: to prove the base case I use the fact that n + 0 = n and substitute n with 0
step3: base case is done
step4: use the fact that n + 0 = n and use the commutative property of addition to show that 0 + succ(k) = succ(k)
step5: hence we are done
"""

wrong_steps_2 = """
problem: theorem zero_add_thm (k: ℕ) : 0 + k = k
step1: This can be written as 0 + k * 1 = k
step2: This can be written as 0 = k - k * 1
step3: since k = k * 1, k - k * 1 = 0
step4: hence 0 = 0 and we're done
"""


## Tactic Indices

In [199]:
test_structured = ask_llm_structured(steps)
for i, step in enumerate(test_structured.steps):
    print(f"Step {i + 1}: Tactic: {step.tactic_template}, Theorem: {step.theorem_name}")

Step 1: Tactic: 3, Theorem: None
Step 2: Tactic: 1, Theorem: None
Step 3: Tactic: 4, Theorem: None
Step 4: Tactic: 1, Theorem: None
Step 5: Tactic: 5, Theorem: None
Step 6: Tactic: 4, Theorem: None


## Proof in Lean

In [186]:
complete_tactics = []
for step in test_structured.steps:
    if step.theorem_name is not None:
        complete_tactic = tactics[step.tactic_template - 1].replace("name_of_theorem", theorems[step.theorem_name - 1], 1)
        complete_tactics.append(complete_tactic)
    else:
        complete_tactics.append(tactics[step.tactic_template - 1])

tactic_steps = [f"""    {step} """ for step in complete_tactics]
tactic_steps = "\n".join(tactic_steps)
print(tactic_steps)


    induction k with d hd 
    rw [add_zero] 
    rfl 
    rw [add_zero] 
    rw [add_comm] 
    rfl 


## Compile

In [184]:
lean_compile(f'''import Game.Levels.Addition.L04add_assoc
namespace MyNat



theorem zero_add_thm (k: ℕ) : 0 + k = k := by
{tactic_steps}
'''
);

In [256]:
def pipeline(statement):
    structured_steps = ask_llm_structured(statement)
    tactics = [f"""    {step.Lean} """ for step in structured_steps.steps]
    tactics = "\n".join(tactics)
    output = lean_compile(f'''import Game.Levels.Addition.L04add_assoc
namespace MyNat
theorem zero_add_thm (k: ℕ) : 0 + k = k := by
{tactics}
'''
)
    if(output == False):
        return tactics, "The proof is correct"
    tutoring_out = ask_llm_tutor_structured(output, statement, tactics)
    return tactics, tutoring_out


In [257]:
output, t_out = pipeline(steps)

In [253]:
print(output)

    induction k with 0 hd 
    rw [add_zero] 
    rfl 
    rw [add_succ] 
    hd 
    rfl 


In [215]:
print(f''' you have an error in step {t_out.line - 3} and the tutor says {t_out.tutor}''')

 you have an error in step 5 and the tutor says The error you're encountering is due to the way you've structured your proof. Lean requires that every recursive step in your proof must be based on a smaller or simpler case. In your proof, it seems that the base case isn't being recognized as a proper starting point for the induction, which is causing the termination issue. To fix this, you can try using 'have' statements to break down your goals into simpler parts, or you might need to specify a different method for proving the induction step. This will help Lean understand how your proof progresses.


## not structured

In [135]:
output = ask_llm(steps)

In [136]:
print(output)

induction k with d hd  
rw [add_succ]  
rfl  
rw [add_succ]  
rw [add_zero]  
rfl  


In [137]:
tactics = output.split("\n")
# add tab to each tactic
tactics = [f"    {tactic}" for tactic in tactics]
tactics = "\n".join(tactics)
print(tactics)

    induction k with d hd  
    rw [add_succ]  
    rfl  
    rw [add_succ]  
    rw [add_zero]  
    rfl  


In [138]:
lean_compile(f'''import Game.Levels.Addition.L04add_assoc
namespace MyNat



theorem zero_add_thm (k: ℕ) : 0 + k = k := by
{tactics}
'''
)

"Compilation Error:\n/Users/arnavmehta/Desktop/AIInEd/NNG4/Game/Levels/Addition/temp_theorem.lean:10:8: error: tactic 'rewrite' failed, did not find instance of the pattern in the target expression\n  ?a + succ ?d\ncase zero\n⊢ 0 + 0 = 0\n\n"